In [8]:
import pandas as pd
import numpy as np

import os
cwd = os.getcwd()
kaggle = cwd == "/kaggle/working"

# pretrain = pd.read_parquet("pretrain.parquet")
# train = pd.read_parquet("train.parquet")
# test = pd.read_parquet("test.parquet")

pretrain = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "pretrain.parquet")
train = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "train.parquet")
test = pd.read_parquet(("/kaggle/input/latsis-experiments/" if kaggle else "") + "test.parquet")

#convert to string
train["text"] = train["text"].astype(str)
test["text"] = test["text"].astype(str)

#keep only the first 1000 rows
# pretrain = pretrain[:1000]
# train = train[:100]
test = test[:2500]

In [9]:
train["label"].sum()/len(train)

0.32650414937759337

In [10]:
import torch
from torch import nn
from transformers import TrainingArguments, Trainer, AutoTokenizer, XLMRobertaTokenizerFast, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainerCallback, TrainerControl, TrainingArguments
# model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)
# tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/mdeberta-v3-base", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")
# model = AutoModelForSequenceClassification.from_pretrained('aari1995/German_Semantic_STS_V2', num_labels=1)
# tokenizer = AutoTokenizer.from_pretrained('aari1995/German_Semantic_STS_V2')
# model = AutoModelForSequenceClassification.from_pretrained("deepset/gelectra-base", num_labels=1, ignore_mismatched_sizes=True)
# tokenizer = AutoTokenizer.from_pretrained("deepset/gelectra-base")
config = model.config
tokenizer.model_max_length = config.max_position_embeddings
# model.classifier.out_proj = nn.Sequential(
#     nn.Linear(config.hidden_size, 1),
#     nn.Sigmoid()
# )



print("model parameters:" + str(sum(p.numel() for p in model.parameters())))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Hate-speech-CNERG/dehatebert-mono-german and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model parameters:167357185


In [11]:
from torch.utils.data import Dataset
import torch
import numpy as np

max_length = 128

def encode_texts(tokenizer, texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.tensor(input_ids), torch.tensor(attention_masks)

pretrain_x, pretrain_attention_mask = encode_texts(tokenizer, pretrain['text'])
pretrain_y = torch.tensor(np.array(pretrain['label'].tolist()), dtype=torch.float32)

train_x, train_attention_mask = encode_texts(tokenizer, train['text'])
train_y = torch.tensor(np.array(train['label'].tolist()), dtype=torch.float32)

test_x, test_attention_mask = encode_texts(tokenizer, test['text'])
test_y = torch.tensor(np.array(test['label'].tolist()), dtype=torch.float32)

#mix of train and pretrain
mixed_x = torch.cat((train_x, pretrain_x))
mixed_attention_mask = torch.cat((train_attention_mask, pretrain_attention_mask))
mixed_y = torch.cat((train_y, pretrain_y))


class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label = label

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'label': self.label[idx],
        }

pretrain_dataset = CustomDataset(pretrain_x, pretrain_attention_mask, pretrain_y)
train_dataset = CustomDataset(train_x, train_attention_mask, train_y)
val_dataset = CustomDataset(test_x, test_attention_mask, test_y)
mixed_dataset = CustomDataset(mixed_x, mixed_attention_mask, mixed_y)

print(train_x.shape, train_attention_mask.shape, train_y.shape, test_x.shape, test_attention_mask.shape, test_y.shape, pretrain_x.shape, pretrain_attention_mask.shape, pretrain_y.shape, mixed_x.shape, mixed_attention_mask.shape, mixed_y.shape)


torch.Size([7712, 128]) torch.Size([7712, 128]) torch.Size([7712]) torch.Size([2500, 128]) torch.Size([2500, 128]) torch.Size([2500]) torch.Size([19133, 128]) torch.Size([19133, 128]) torch.Size([19133]) torch.Size([26845, 128]) torch.Size([26845, 128]) torch.Size([26845])


In [15]:
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F

model = model.cuda()
optimizer = None



criterion = BCEWithLogitsLoss()

# Training function
def train(model, lr_per_epoch, train_dataset, val_dataset):
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4)

    for epoch in range(len(lr_per_epoch)):
        model.train()
        lr = lr_per_epoch[epoch]
        optimizer.param_groups[0]['lr'] = lr
        train_loss = 0.0

        # Training loop with tqdm
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            outputs = model(inputs, attention_mask=attention_mask)
            logits = outputs.logits.squeeze(-1)  # Remove the last dimension

            loss = criterion(logits, labels)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            
            train_loss += loss.item()
            
        avg_train_loss = train_loss / len(train_loader)
        

        # Validation loop
        model.eval()
        val_loss = 0.0
        all_predictions_raw = []
        all_labels = []

        # Validation loop with tqdm
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            inputs = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            labels = batch['label'].cuda()

            with torch.no_grad():
                outputs = model(inputs, attention_mask=attention_mask, labels=labels)
                logits = outputs.logits.squeeze(-1)
                val_loss += criterion(logits, labels).item()
                pred = F.sigmoid(logits)
                
                all_predictions_raw.append(pred.cpu())
                all_labels.append(labels.cpu())

        avg_val_loss = val_loss / len(val_loader)
        

        all_predictions_raw = torch.cat(all_predictions_raw)
        all_labels = torch.cat(all_labels)
        accuracy = accuracy_score(all_labels.numpy(), all_predictions_raw.numpy() >= 0.5)
        print(f"Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Accuracy: {accuracy}")
        # test different treshholds
        for tresh in np.arange(0.2, 0.75, 0.05):
            f1 = f1_score(all_labels.numpy(), all_predictions_raw.numpy() >= tresh, average='macro')
            print(f"tresh: {tresh}, f1: {f1}")

        

        # print(f"First predictions:")
        # i = 0
        # for y_pred, y in zip(all_predictions_raw, all_labels):
        #     print(f"y_pred: {y_pred.item()}, y: {y}")
        #     i += 1
        #     if i > 5:
        #         break

        print("\n")

# Train the model

# for param in model.parameters():
#     param.requires_grad = False
# for param in model.classifier.parameters():
#     param.requires_grad = True
# for i in range(-3, 0):
#     for param in model.roberta.encoder.layer[i].parameters():
#         param.requires_grad = True
# optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=1e-5)

# train(model, [1e-9, 1e-5, 1e-5, 1e-5], mixed_dataset, val_dataset)



for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True
for i in range(-3, 0):
    # for param in model.roberta.encoder.layer[i].parameters():
    for param in model.bert.encoder.layer[i].parameters():
    # for param in model.deberta.encoder.layer[i].parameters():
    # for param in model.electra.encoder.layer[i].parameters():
        param.requires_grad = True
optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=1e-5)

train(model, [1e-9, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-6], train_dataset, val_dataset)

# Save the model
torch.save(model, '/kaggle/working/model.pt' if kaggle else 'model.pt')
torch.save(tokenizer, '/kaggle/working/tokenizer.pt' if kaggle else 'tokenizer.pt')
print("saved")

c:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1:   0%|          | 0/1928 [00:00<?, ?it/s]

Training Epoch 1:   0%|          | 7/1928 [00:03<14:28,  2.21it/s]  

Training Epoch 1:   2%|▏         | 42/1928 [00:05<01:30, 20.73it/s]

In [14]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a